In [30]:
!pip install -q -U kaggle_environments

In [31]:
import random
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [32]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [33]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [34]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [35]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

In [36]:
%%writefile paper_agent.py

# №1 агент всегда выбирает бумагу
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [37]:
%%writefile scissors_agent.py

# №2 агент всегда выбирает ножницы
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [38]:
%%writefile random_agent.py

# №3 агент выбирает случайный ход
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


In [39]:
%%writefile cycle_agent.py

# введение переменной для фиксации последнего хода
last_action = -1

# №4 агент выбирает ходы в циклическом порядке 0 - 1 - 2
def your_agent(observation, configuration):
    global last_action
    last_action = (last_action + 1) % 3
    return last_action

Overwriting cycle_agent.py


In [40]:
%%writefile win_opponent.py

# №5 агент выбирает ход, который бьет предыдущий ход противника
def win_opponent(observation, configuration):
    # действие, если это не первый розыгрыш
    if observation.step > 0:
        # записываем последний ход противника
        last_opponent_action = observation.lastOpponentAction
        # выбираем ход, который побеждает предыдущий ход противника и сбрасываем его
        return (last_opponent_action + 1) % configuration.signs
    else:
        # действие, если это первый розыгрыш - выбираем случайный ход
        return random.randrange(0, configuration.signs)

Overwriting win_opponent.py


In [41]:
%%writefile sequence_agent.py

# №6 агент, повторяющий последовательность 0, 2, 2
#0 - rock
#1 - paper
#2 - scissors
def sequence_agent(observation, configuration):
    # введение переменной для определения последовательности
    sequence = [0, 2, 2]
    # выбираем текущий ход на основе номера шага (поделим номер шага на длину последовательности)
    return sequence[observation.step % len(sequence)]

Overwriting sequence_agent.py


In [42]:
%%writefile sequence_agent2.py

# №7 агент, повторяющий последовательность 2, 0, 1, 0, 2
#0 - rock
#1 - paper
#2 - scissors
def sequence_agent2(observation, configuration):
    # введение переменной для определения последовательности
    sequence = [2, 0, 1, 0, 2]
    # выбираем текущий ход на основе номера шага (поделим номер шага на длину последовательности)
    return sequence[observation.step % len(sequence)]

Overwriting sequence_agent2.py


In [43]:
%%writefile self_win_agent.py

# введение переменной для фиксации последнего хода
last_action1 = -1

# №8 агент выбирает ход, побеждающий его собственный предыдущий
def self_win_agent(observation, configuration):
    # используем переменную для хранения последнего хода
    global last_action1

    # если это первый розыгрыш - выбираем случайный ход
    if observation.step == 0:
        last_action1 = random.randrange(0, configuration.signs)
    else:
        # если розыгрыш не первый, то выбираем "жест", побеждающий свой предыдущий ход
        last_action1 = (last_action1 + 1) % configuration.signs

    return last_action1

Overwriting self_win_agent.py


In [44]:
%%writefile probability_agent.py

# №9 агент, который запоминает историю ходов соперника и расчитывает вероятности, основываясь на которой выбирает оптимальный "жест"
def probability_agent(observation, configuration):
    # если первый розыгрыш
    if observation.step == 0:
        # то создаем массив для подсчета частот ходов соперника
        global opponent_moves_count
        opponent_moves_count = [0] * configuration.signs
    else:
        # далее по ходам увеличиваем счетчик последнего хода соперника
        opponent_moves_count[observation.lastOpponentAction] += 1

    # вычисляем вероятность каждого "жеста" соперника
    total_moves = sum(opponent_moves_count)
    probabilities = [count / total_moves for count in opponent_moves_count]

    # выбираем "жест", который побеждает наиболее вероятный "жест" соперника
    most_probable_action = probabilities.index(max(probabilities))
    best_response = (most_probable_action + 1) % configuration.signs

    return best_response

Overwriting probability_agent.py


In [45]:
%%writefile weighted_random_agent.py

# №10 агент выбирает жесты  в соответствии с заданными вероятностями
def weighted_random_agent(observation, configuration):
    # указываем вероятности для каждого жеста
    probabilities = [0.3, 0.2, 0.5]  # 30% камень, 20% бумага, 50% ножницы

    # используем random.choices для выбора на основе вероятностей
    return random.choices(population=[0, 1, 2], weights=probabilities, k=1)[0]


Overwriting weighted_random_agent.py


In [46]:
%%writefile time_based_agent.py

import time

# №11 агент выбирает ход на основе последней цифры системного времени
def time_based_agent(observation, configuration):
    # получаем текущее время в секундах
    current_time = time.time()

    # отсекаем и записываем в переменную последнюю цифру текущего времени (в секундах)
    last_digit = int(str(int(current_time))[-1])

    # делим последнюю цифру на 3 и выбираем ход в зависимости от остатка
    return last_digit % 3


Overwriting time_based_agent.py


In [47]:
%%writefile repeat_winner_agent.py

# №12 агент, который посторяет "жест" победителя
def repeat_winner_agent(observation, configuration):
    # если первый розыгрыш, то выбираем рандомный "жест"
    if observation.step == 0:
        return random.randrange(0, configuration.signs)

    # определяем, кто выиграл в предыдущем ходе
    # выигравший: 0 - камень, 1 - бумага, 2 - ножницы
    # определяем победителя по результату предыдущего хода
    if observation.lastRoundWinner == 0:
        # если победил первый игрок, агент выбирает его "жест"
        return observation.lastOpponentAction
    else:
        # если победил второй игрок, агент выбирает его "жест"
        return observation.lastOpponentAction


Overwriting repeat_winner_agent.py


In [48]:
# создадим список с наименованиеями всех агентов (для случаев, когда наименование оканчивается на .py - включить в список)
agents = [case for case in os.listdir() if case.endswith(".py")]
print(agents)

['paper_agent.py', 'self_win_agent.py', 'time_based_agent.py', 'copy_opponent.py', 'rock_agent.py', 'random_agent.py', 'cycle_agent.py', 'repeat_winner_agent.py', 'weighted_random_agent.py', 'sequence_agent2.py', 'win_opponent.py', 'probability_agent.py', 'sequence_agent.py', 'scissors_agent.py']


In [49]:
# создадим словарь, содержащий настройки турнира
tournament = {"episodeSteps": 100}  # задаем число сбросов в паре агентов

# создаем словарь для подсчёта побед
win_count = {agent: 0 for agent in agents}

# создаем список для хранения результатов игр
game_results = []

# запускаем порядок игр
# перебираем всех агентов в списке, где i - индекс первого агента, а agent1 - имя первого агента в текущей паре
for i, agent1 in enumerate(agents):
    # перебираем всех агентов в списке для создания пары, где j - индекс второго агента, а agent2 - имя второго агента в текущей паре
    for j, agent2 in enumerate(agents):
        # введем условие для избегания повторных пар и предотвращения игры с самим собой
        if i < j:
            # запускаем игру между двумя выбранными агентами с указанными настройками турнира
            result = evaluate("rps", [agent1, agent2], configuration=tournament)

            # считаем колво очков (результат функции evaluate)
            score1, score2 = result[0]  # очки первого и второго агентов

            # введем проверку для избегания операций с None, если одно из значений равно None то присваивем проигрыш
            if score1 is None or score2 is None:
                if score1 is None:
                    score1 = 0
                if score2 is None:
                    score2 = 0

            # определяем победителя, если score1 больше score2, тогда выигрывает агент1, если наоброт, то агент2, если же иная ситуация - то ничья
            if score1 > score2:
                winner = agent1
            elif score2 > score1:
                winner = agent2
            else:
                winner = "ничья"

            # записываем результаты игры в словарь
            game_results.append({
                "agent1": agent1,
                "agent2": agent2,
                "score1": score1,
                "score2": score2,
                "winner": winner
            })

            # увеличиваем счётчик побед, добавив +1 победившему агенту в словарь win_count, а если ничья - то не производим действия
            if winner != "ничья":
                win_count[winner] += 1


In [50]:

# выведем статистику
print("\nРезультаты игр между агентами:")
for game in game_results:
    # выводим словари-результаты, записанные в список game_results
    print(
        f"{game['agent1']} против {game['agent2']}, "
        f"Счет: {game['score1']} - {game['score2']}, "
        f"Победитель: {game['winner']}"
    )


Результаты игр между агентами:
paper_agent.py против self_win_agent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против time_based_agent.py, Счет: 99.0 - -99.0, Победитель: paper_agent.py
paper_agent.py против copy_opponent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против rock_agent.py, Счет: 99.0 - -99.0, Победитель: paper_agent.py
paper_agent.py против random_agent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против cycle_agent.py, Счет: 0 - 0, Победитель: ничья
paper_agent.py против repeat_winner_agent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против weighted_random_agent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против sequence_agent2.py, Счет: 0 - 0, Победитель: ничья
paper_agent.py против win_opponent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против probability_agent.py, Счет: 1 - 0, Победитель: paper_agent.py
paper_agent.py против sequence_agent.py, Счет: -33.0 - 33.0, Победитель: se

In [51]:

# ранжируем список подсчета побед
sorted_win_count = sorted(win_count.items(), key=lambda x: x[1], reverse=True)

# выводим итоговый список подсчета побед
print("\nИтоговое количество побед каждого агента:")
for agent, wins in sorted_win_count:
    print(f"{agent}: {wins} побед")


Итоговое количество побед каждого агента:
rock_agent.py: 10 побед
sequence_agent.py: 10 побед
paper_agent.py: 9 побед
time_based_agent.py: 9 побед
sequence_agent2.py: 9 побед
scissors_agent.py: 8 побед
cycle_agent.py: 7 побед
self_win_agent.py: 0 побед
copy_opponent.py: 0 побед
random_agent.py: 0 побед
repeat_winner_agent.py: 0 побед
weighted_random_agent.py: 0 побед
win_opponent.py: 0 побед
probability_agent.py: 0 побед
